# Optimal String Configuration

## Summary

The goal of the optimal string configuration is to produce an ideal stringing configuration for a string-inverter architecture of solar array, taking into account constraints on string length and number of strings per MPPT.

## Problem Statement

The stringing problem can be formulated mathematically as a mixed-integer linear programming problem using the following model:

### Sets

 $F$ = set of potential string lengths  
 $M$ = number of strings per mppt
 

### Parameters

 $f_i$ = number of panels in string length $i$, $\forall i \in F$  
 $m_j$ = number of strings $j$ per MPPT, $\forall j \in M$  
 $P_{max}$ = number of panels available  
 $p_r$ = maximum number of panels by which the total panel count can be relaxed  
 
 
### Variables
 $x_i$ = number of strings $i$ to deploy $\forall i \in F$  
 $g_ij$ = number of mppt groupings $j$ to deploy $\forall j \in M, \forall i \in F$
 

### Objective

Minimize the total number of strings deployed  
 $\min \sum_{i \in F} x_i$

### Constraints

Limit the number of panels to be below $P_{max}$  
 $P_{max} > \sum_{i \in F} x_i f_i$

Make sure we don't reduce the panel count by more than $p_r$  
 $P_{max} - p_r < \sum_{ i \in F} x_i f_i$
 
Number of strings is a positive integer
 $f_i \geq 0, f_i \in \{0,1\}^{n_y}  \forall i \in F$

Number of MPPT groupings is a positive integer
 $g_{i,j} \geq 0, g_{i,j} \in \{0,1\}^{n_y}  \forall j \in M, \forall i \in F$

Link $x_i$ and $g_i$ via linear equations  
 $x_i = \sum_{j \in M} m_j g_{i,j}, \forall i \in F$  
 
 
### Example

Suppose that
* the valid string lengths are determined to be 17 or 18 panels per string, based on the acceptable minimum and maximum voltage tolerances for each string, such that $F = [f_{17}, f_{18}] = [17,18]$.
* the valid number of strings per mppt is 3 or 2, such that $M = [m_2,m_3] = [2,3]$.
* the layout accomodates a maximum of $P_{max} = 1030$ panels
* we can tolerate a relaxation of up to $p_r = 5$ panels



The goal of our minimization problem is to find the optimal string configuration to minimize stringing.

The total number of panels will be $P = f_{17} x_{17} + f_{18} x_{18} = 17 x_{17} + 18 x_{18}$  The objective function is $\min x_{17}+x_{18}$ - in other words, we want to find the lowest numbers of $x_{17}$ and $x_{18}$ that satisfy our set of constraints.

The constraints are as follows:
* $P < 1030$, or, the total number of panels can't be higher than $P_{max}$, which is what the layout accomodates.
* $P > P_{max} - p_r = 1025$, or, we can't remove more than 5 panels to achieve our goal
* limit the possible values for $x_{17}$, $x_{18}$ such that strings can be divided into groups of 2 or 3h
    * $x_{18} = m_2 g_{2,18} + m_3 g_{3,18} = 2 g_{2,18} + 3 g_{3,18}$ where g_2 and g_3 are integers --- in other words, $x_{18}$ must be divisible by a combination of groupings of 2 and/or 3.
    * $x_{17} = 2 g_{2,17} + 3 g_{3,17}$
    
When we run this MILP through a solver, it produces the following result:
* $x_{17} = 14$, $x_{18} = 44$
* $g_{2,17} = 1, g_{3,17} = 4, g_{2,18} = 4, g_{3,18} = 22$






## Implementation

With python and puyomo, the implementation of this algorithm as a solver is extremely straightforward. First we import pyomo and instantiate a model:

In [7]:
from pyomo.environ import *

model = AbstractModel()

ImportError: No module named 'pyomo'

We define our two sets, $F$ and $M$, which represent the main parameters associated with each variable.

In [2]:

# set of potential string lengths
model.F = Set()
# potential strings per MPPT
model.M = Set()



NameError: name 'Set' is not defined

We then define the parameter values as follows:

In [ ]:

#number of panels per string length
model.f = Param(model.F, within=NonNegativeIntegers)

#number of strings per mppt grouping
model.m = Param(model.M, within=NonNegativeIntegers)

# max number of panels
model.Pmax = Param(within=NonNegativeIntegers)
# max number of panels we can remove
model.p_r = Param(within=NonNegativeIntegers)

The above lines of code will assign values to $f$, $m$, $P_{max}$ and $p_r$ based on the parameters defined in the model data file.

We then define our variables:

In [ ]:
# variables x and g
model.x = Var(model.F, within=NonNegativeIntegers)
model.g = Var(model.F, model.M, within=NonNegativeIntegers)

Our objective function is as follows:

In [ ]:
# define the objective function
def cost_rule(model):
    return sum(model.x[i] for i in model.F)
model.cost = Objective(rule=cost_rule)

Constraints:


In [ ]:
# define Pmax Constraint
def panel_limit(model):
    return model.Pmax >= sum(model.f[i]*model.x[i] for i in model.F)
model.max_panel_count = Constraint(model.F, rule=panel_limit)

# define p_r relaxation constraint
def panel_relaxation(model):
    return model.Pmax-model.p_r <= sum(model.f[i]*model.x[i] for i in model.F)
model.max_panel_relaxation = Constraint(model.F, rule=panel_relaxation)

# link g_ij and x_i via M to get MPPT sizing
def mppt_string_count(model, i):
    return model.x[i] == sum(model.m[j]*model.g[i,j] for j in model.M)
model.mppt_grouping = Constraint(model.F, rule=mppt_string_count)

In [5]:
%cat stringing_ex.py

ERROR:root:Line magic function `%cat` not found.
